# 语义分割模型使用实例——道路分割

首先导入必要的包

In [1]:
from dataset_preparation import linestrip_to_mask, split_image_mask_train, split_train_val_test, visualize_mask
from training import train
from inference import inference, skeleton
import requests
import zipfile
import os

ModuleNotFoundError: No module named 'lightning'

运行以下代码下载数据集

In [ ]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

id = "1oFHu6KEzEHMrnKwSJ8iye4Ggp1BZ-czD"
destination = "road_network.zip"
download_file_from_google_drive(id, destination)

zip_path = "road_network.zip"
extract_path = "./road_network_detection/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        file_name = file_info.filename

        # Attempt to decode the file name
        try:
            file_name = file_name.encode('cp437').decode('gbk')  # Adjust the encoding as needed
        except:
            file_name = file_name.encode('cp437').decode('utf-8', 'ignore')  # Fallback to UTF-8

        extracted_path = os.path.join(extract_path, file_name)
        
        # Create directory structure
        if file_info.is_dir():
            os.makedirs(extracted_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(extracted_path), exist_ok=True)
            
            # Extract and write the file
            with open(extracted_path, 'wb') as f:
                f.write(zip_ref.read(file_info.filename))